In [ ]:
import os
from skimage import io
import numpy as np
import mahotas as mh
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from scipy import optimize
import scipy.stats
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from skimage.filters import meijering
import skimage.filters as filt
from skimage.morphology import skeletonize

pxperum=6.8969

In [ ]:
def FLAT(X):
    return [item for sublist in X for item in sublist]

def clean_line(x,y):
    dist=np.sqrt(np.diff(x)**2+np.diff(y)**2)
    i=0
    while np.max(dist>2):
        print(i)
        if dist[i]>2:
            x_new=np.concatenate((x[0:i+1],x[i+2:]))
            y_new=np.concatenate((y[0:i+1],y[i+2:]))
            new_dist=np.zeros(len(x_new))
            for ii in range(0,len(x_new)):
                x_new2=np.concatenate((np.append(x_new[0:ii],x[i+1]),x_new[ii:]))
                y_new2=np.concatenate((np.append(y_new[0:ii],y[i+1]),y_new[ii:]))
                new_dist[ii]=np.sum(np.sqrt(np.diff(x_new2)**2+np.diff(y_new2)**2))
            ii_fin=np.argmin(new_dist)
            x=np.concatenate((np.append(x_new[0:ii_fin],x[i+1]),x_new[ii_fin:]))
            y=np.concatenate((np.append(y_new[0:ii_fin],y[i+1]),y_new[ii_fin:]))
            dist=np.sqrt(np.diff(x)**2+np.diff(y)**2)
            i=0
        else:
            i=i+1
    
    return x,y   

method_2 = "leastsq"

def calc_R(xc, yc):
    return np.sqrt((x1-xc)**2 + (y1-yc)**2)

def f_2(c):
    Ri = calc_R(*c)
    return Ri - Ri.mean()

def endPoints(skel):
    endpoint1=np.array([[0, 0, 0],[0, 1, 0],[2, 1, 2]])
    endpoint2=np.array([[0, 0, 0],[0, 1, 2],[0, 2, 1]])
    endpoint3=np.array([[0, 0, 2],[0, 1, 1],[0, 0, 2]])
    endpoint4=np.array([[0, 2, 1],[0, 1, 2],[0, 0, 0]])
    endpoint5=np.array([[2, 1, 2],[0, 1, 0],[0, 0, 0]])
    endpoint6=np.array([[1, 2, 0],[2, 1, 0],[0, 0, 0]])
    endpoint7=np.array([[2, 0, 0],[1, 1, 0],[2, 0, 0]])
    endpoint8=np.array([[0, 0, 0],[2, 1, 0],[1, 2, 0]])
    ep1=mh.morph.hitmiss(skel,endpoint1)
    ep2=mh.morph.hitmiss(skel,endpoint2)
    ep3=mh.morph.hitmiss(skel,endpoint3)
    ep4=mh.morph.hitmiss(skel,endpoint4)
    ep5=mh.morph.hitmiss(skel,endpoint5)
    ep6=mh.morph.hitmiss(skel,endpoint6)
    ep7=mh.morph.hitmiss(skel,endpoint7)
    ep8=mh.morph.hitmiss(skel,endpoint8)
    ep = ep1+ep2+ep3+ep4+ep5+ep6+ep7+ep8
    return ep

def CenterOfMass(im):
    im=filt.gaussian(im,sigma=1)
    thresh = filt.threshold_otsu(im)
    im=im>thresh
    x,y=np.where(im>0)
    return np.mean(x),np.mean(y)

In [ ]:
halfsize=7

In [ ]:
#Quantify curvature from single cells, each cell is in a separate folder together with a MASK which is a sinlge 
#pixel width outline of the membrane to be quantified
show=0
curv1=[]
nr_of_cells= #input number of cells
path0=#input path to folder containing cell folders
for j in tqdm(range(nr_of_cells)):
    curv=[]
    path=path0+'/Cell'+str(j+1)+'/'
    files=os.listdir(path)
    ind=np.array(['MASK' in i for i in files])
    im=io.imread(path+np.array(files)[ind][0])
    x,y=np.where(im>0)

    ex,ey=np.where(endPoints(im>0))

    new_x=[]
    new_y=[]

    dist=np.sqrt((x-ex[0])**2+(y-ey[0])**2)
    new_x.append(x[np.argmin(dist)])
    new_y.append(y[np.argmin(dist)])
    x=np.delete(x,np.argmin(dist))
    y=np.delete(y,np.argmin(dist))

    while len(x)>0:
        dist=np.sqrt((x-new_x[-1])**2+(y-new_y[-1])**2)
        new_x.append(x[np.argmin(dist)])
        new_y.append(y[np.argmin(dist)])
        x=np.delete(x,np.argmin(dist))
        y=np.delete(y,np.argmin(dist))

    new_x=np.array(new_x)
    new_y=np.array(new_y)
    
    new_x,new_y=clean_line(new_x,new_y)
    
    ln_full=len(new_x)
    
    dst=np.sqrt((new_x[0:-1]-new_x[1:])**2+(new_y[0:-1]-new_y[1:])**2)
    
    ln=0
    while np.max(dst)>2:
        new_x=new_x[:-1]
        new_y=new_y[:-1]
        dst=np.sqrt((new_x[0:-1]-new_x[1:])**2+(new_y[0:-1]-new_y[1:])**2)
        ln+=1
    
    print(str(j)+': '+str(ln_full)+'_'+str(ln))
    
    
    im2=io.imread(path+np.array(files)[~ind][0])
    im2=im2[1,:,:]
    
    fid1,fid2=CenterOfMass(im2)

    new_x=np.append(new_x,[fid1],0)
    new_y=np.append(new_y,[fid2],0)

    new_x=np.flip(new_x)
    new_y=np.flip(new_y)

    new_x=np.append(new_x,[fid1],0)
    new_y=np.append(new_y,[fid2],0)

    cell_polygon=[]

    for iii in range(len(new_x)):
        cell_polygon.append((new_x[iii],new_y[iii]))

    polygon = Polygon(cell_polygon)
    
    for i in range(halfsize+1,len(new_x)-halfsize-1):
        x1=new_x[i-halfsize:i+halfsize+1]
        y1=new_y[i-halfsize:i+halfsize+1]    
        center_estimate = np.mean(x1), np.mean(y1)
        center_2, ier = optimize.leastsq(f_2, center_estimate)
        
        xv=center_2[0]-new_x[i]
        yv=center_2[1]-new_y[i]
        n=np.sqrt(xv**2+yv**2)
        xv=xv/n/10
        yv=yv/n/10

        point = Point(new_x[i]+xv,new_y[i]+yv)
        Ri_2  = calc_R(*center_2)
        if polygon.contains(point):
            curv.append(-1/Ri_2.mean()*pxperum)
        else:
            curv.append(1/Ri_2.mean()*pxperum)
    
    curv1.append(curv)
    
    if show==1:
        ratio=(np.max(new_x)-np.min(new_x))/(np.max(new_y)-np.min(new_y))
        plt.figure(figsize=[5*ratio,5])
        plt.set_cmap('Greys')
        plt.scatter((new_x-np.max(new_x)/2-np.min(new_x)/2)/100,(new_y-np.max(new_y)/2-np.min(new_y)/2)/100,c=res*pxperum,alpha=1,s=4)
        plt.tick_params(labelsize=12)
        plt.xlabel('x [$\mu m$]',fontsize=13)
        plt.ylabel('y [$\mu m$]',fontsize=13)
        cbar=plt.colorbar()
        cbar.set_label('|C| [1/$\mu m$]',fontsize=13)
        plt.title(np.array(files)[ind][0])
        plt.gca().set_aspect('equal')
        plt.pause(0.1)

In [ ]:
#Quantify curvature from single cells, each cell is in a separate folder together with a MASK which is a sinlge 
#pixel width outline of the membrane to be quantified
show=0
curv2=[]
nr_of_cells= #input number of cells
path0=#input path to folder containing cell folders
for j in tqdm(range(nr_of_cells)):
    curv=[]
    path=path0+'/Cell'+str(j+1)+'/'
    files=os.listdir(path)
    ind=np.array(['MASK' in i for i in files])
    im=io.imread(path+np.array(files)[ind][0])
    x,y=np.where(im>0)

    ex,ey=np.where(endPoints(im>0))

    new_x=[]
    new_y=[]

    dist=np.sqrt((x-ex[0])**2+(y-ey[0])**2)
    new_x.append(x[np.argmin(dist)])
    new_y.append(y[np.argmin(dist)])
    x=np.delete(x,np.argmin(dist))
    y=np.delete(y,np.argmin(dist))

    while len(x)>0:
        dist=np.sqrt((x-new_x[-1])**2+(y-new_y[-1])**2)
        new_x.append(x[np.argmin(dist)])
        new_y.append(y[np.argmin(dist)])
        x=np.delete(x,np.argmin(dist))
        y=np.delete(y,np.argmin(dist))

    new_x=np.array(new_x)
    new_y=np.array(new_y)
    
    new_x,new_y=clean_line(new_x,new_y)
    
    ln_full=len(new_x)
    
    dst=np.sqrt((new_x[0:-1]-new_x[1:])**2+(new_y[0:-1]-new_y[1:])**2)
    
    ln=0
    while np.max(dst)>2:
        new_x=new_x[:-1]
        new_y=new_y[:-1]
        dst=np.sqrt((new_x[0:-1]-new_x[1:])**2+(new_y[0:-1]-new_y[1:])**2)
        ln+=1
    
    print(str(j)+': '+str(ln_full)+'_'+str(ln))
    
    res=new_x*0.0
    for i in range(halfsize,len(new_x)-halfsize):
        x1=new_x[i-halfsize:i+halfsize+1]
        y1=new_y[i-halfsize:i+halfsize+1]    
        center_estimate = np.mean(x1), np.mean(y1)
        center_2, ier = optimize.leastsq(f_2, center_estimate)
        Ri_2  = calc_R(*center_2)
        curv.append(1/Ri_2.mean()*pxperum)
        curv.append(-1/Ri_2.mean()*pxperum)
    
    
    curv2.append(curv)
    
    if show==1:
        ratio=(np.max(new_x)-np.min(new_x))/(np.max(new_y)-np.min(new_y))
        plt.figure(figsize=[5*ratio,5])
        plt.set_cmap('Greys')
        plt.scatter((new_x-np.max(new_x)/2-np.min(new_x)/2)/100,(new_y-np.max(new_y)/2-np.min(new_y)/2)/100,c=res*pxperum,alpha=1,s=4)
        plt.tick_params(labelsize=12)
        plt.xlabel('x [$\mu m$]',fontsize=13)
        plt.ylabel('y [$\mu m$]',fontsize=13)
        cbar=plt.colorbar()
        cbar.set_label('|C| [1/$\mu m$]',fontsize=13)
        plt.title(np.array(files)[ind][0])
        plt.gca().set_aspect('equal')
        plt.savefig(np.array(files)[ind][0]+'_plot.svg')
        plt.pause(0.1)

In [ ]:
Single=FLAT(curv1)
Double=FLAT(curv2)

In [ ]:
plt.figure(figsize=[5,5])

countS,bins=np.histogram(Single,np.linspace(-2,2,32)) 
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.bar(bincenters,countS/np.sum(countS),bins[1]-bins[0],color='tab:green',alpha=0.9)

countD,bins=np.histogram(Double,np.linspace(-2,2,32)) 
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.bar(bincenters,countD/np.sum(countD),bins[1]-bins[0],color='tab:grey',alpha=0.9)

plt.bar(bincenters,np.minimum(countS/np.sum(countS),countD/np.sum(countD)),bins[1]-bins[0],color='k')

plt.xlabel('C [$\mu m^{-1}$]')
plt.ylabel('fraction')
plt.legend(['cell singles','cell  doubles'])

In [ ]:
plt.figure(figsize=[5,5])

countD,bins=np.histogram(Single,np.linspace(-2,2,32))
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.bar(bincenters,countD/np.sum(countD),bins[1]-bins[0],color='tab:green',alpha=0.5)

for i in range(len(curv1)):
    count,bins=np.histogram(curv1[i],np.linspace(-2,2,32)) # displays "world"
    bincenters = 0.5*(bins[1:]+bins[:-1])
    plt.plot(bincenters,count/np.sum(count),'.',color='tab:green')
    
plt.xlabel('C [$\mu m^{-1}$]')
plt.ylabel('fraction')
plt.title('singles')

In [ ]:
plt.figure(figsize=[5,5])

countD,bins=np.histogram(Double,np.linspace(-2,2,32))
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.bar(bincenters,countD/np.sum(countD),bins[1]-bins[0],color='tab:gray',alpha=0.5)

for i in range(len(curv1)):
    count,bins=np.histogram(curv2[i],np.linspace(-2,2,32)) # displays "world"
    bincenters = 0.5*(bins[1:]+bins[:-1])
    plt.plot(bincenters,count/np.sum(count),'.',color='tab:gray')
    
plt.xlabel('C [$\mu m^{-1}$]')
plt.ylabel('fraction')
plt.title('doubles')

In [ ]:
plt.figure(figsize=[5,5])

countD,bins=np.histogram(Double,np.linspace(-2,2,32))
countS,bins=np.histogram(Single,np.linspace(-2,2,32))
bincenters = 0.5*(bins[1:]+bins[:-1])
YYY=(countD/np.sum(countD)-countS/np.sum(countS))
YYY=YYY/np.max(abs(YYY))
plt.bar(bincenters[YYY>0],YYY[YYY>0],bins[1]-bins[0],color='tab:red',alpha=0.9)
plt.bar(bincenters[YYY<=0],YYY[YYY<=0],bins[1]-bins[0],color='tab:blue',alpha=0.9)

plt.xlabel('C [$\mu m^{-1}$]')
plt.ylabel('change in curvature upon collision [norm]')

In [ ]:
C_double=np.array([np.mean(abs(np.array(i))) for i in curv2])
C_single=np.array([np.mean(abs(np.array(i))) for i in curv1])

pval=scipy.stats.mannwhitneyu(C_single,C_double)

pxperum=6.8969
yscale=0.03*pxperum
plt.figure(figsize=[2.5,5])
plt.plot((np.random.rand(len(C_single))-0.5)/5,C_single,'o',color='tab:green')
plt.plot((np.random.rand(len(C_double))-0.5)/5+1,C_double,'o',color='tab:grey')
plt.xlim(-0.5,1.5)
plt.xticks([0,1],['singles','doubles'],fontsize=13)
plt.ylabel(r'$ \langle |C| \rangle$ [1/$\mu m$]',fontsize=13)
plt.plot([0,1],[3*yscale,3*yscale],'k')
plt.text(0.1,3.1*yscale,'p = '+str(np.round(pval.pvalue,3)),fontsize=12)
plt.tick_params(labelsize=12)
plt.ylim(0,3.5*yscale)
plt.title('Half-width '+str(halfsize)+'px')

In [ ]:
bin_input=np.linspace(0,2,10)
plt.figure(figsize=[2.5,5])

y,binEdges=np.histogram(abs(np.array(FLAT(curv1))),bin_input)
bincenters = 0.5*(binEdges[1:]+binEdges[:-1])

width=(bincenters[1]-bincenters[0])

plt.bar(bincenters-width/4,y/np.sum(y),width/2,color='tab:green',label='single cells',alpha=0.6)

y,binEdges=np.histogram(abs(np.array(FLAT(curv2))),bin_input)
bincenters = 0.5*(binEdges[1:]+binEdges[:-1])

plt.bar(bincenters+width/4,y/np.sum(y),width/2,color='tab:grey',label='collided cells',alpha=0.6)

SA=np.expand_dims(bincenters,1)

for i in curv1:
    y,binEdges=np.histogram(abs(np.array(i)),bin_input)
    plt.plot(bincenters-width/4,y/np.sum(y),'.',color='tab:green')
    temp=np.expand_dims(y/np.sum(y),1)
    SA=np.concatenate((SA,temp),1)
    
for i in curv2:
    y,binEdges=np.histogram(abs(np.array(i)),bin_input)
    plt.plot(bincenters+width/4,y/np.sum(y),'.k',color='tab:grey')
    temp=np.expand_dims(y/np.sum(y),1)
    SA=np.concatenate((SA,temp),1)
    
plt.legend(fontsize=11)
plt.ylabel('fraction',fontsize=13)
plt.xlabel(r'$ \langle |C| \rangle$ [1/$\mu m$]',fontsize=13)
plt.tick_params(labelsize=12)
#plt.savefig('LS_HIST_CURVE_plot3_V2.svg')
np.savetxt('Figure4n_CurvSingle1to5Double1to5.dat',SA)